# ポケモンごとに覚える技の本文テキストを生成する

## 準備

In [2]:
import pickle
import pandas as pd

In [3]:
# 全ポケモンのURL一覧
url_list = pickle.load(open('./all_gen_pokemon_urls.pkl', 'rb'))
# タイプ データフレーム
df_type = pd.read_csv('./type.csv')
# アタックデックス データフレーム
df_attackdex = pd.read_csv('./attackdex.csv')
# たまごグループ名
egg_groups_name = pickle.load(open('./egg_groups_name.pkl', 'rb'))
# 持ち物 データフレーム
df_holditem = pd.read_csv('./holditemdex.csv')
# 特性 データフレーム
df_abilirydex = pd.read_csv('./abilirydex.csv')

In [4]:
# 関数定義：レベル技のレベル値を変換する関数
def level_val(level):
    if level == '—':
        return '基本'
    elif level == 'Evolve':
        return '進化時'
        
    return level

In [5]:
from pymongo import MongoClient
client = MongoClient()
db = client['pokemon']

## 実行（レジェンズ）

In [25]:
for data in db.all_gen_pokedex.find():
  
  if not [key for key in data.keys() if 'legendsattacks' in key]:
    continue
  else:
    # レベル技
    level_up_text_list = []
    for attack in data[[key for key in data.keys() if 'legendsattacks' in key][0]]:
      level = level_val(attack['Level'])
      if len(df_attackdex[df_attackdex['English_name'].str.lower() == attack['English_name'].lower()]):
        s_attackdex = df_attackdex[df_attackdex['English_name'].str.lower() == attack['English_name'].lower()]
        name = s_attackdex['Japanese_name'].values[0]
        type_val = s_attackdex['Type'].values[0]
        category = s_attackdex['Category'].values[0]
        power = s_attackdex['Power'].values[0]
      else:
        name = attack['English_name']
        type_val = ''
        category = ''
        power = ''
      level_up_text_list.append(f'|{level}|{name}|{type_val}|{category}|{power}|')
      
    # print('**レベルで覚える技')
    # print('|FILTER:T:100%13%CENTER:|32%CENTER|20%CENTER|CENTER:|CENTER:|c')
    # print('|Lv|技名|タイプ|分類|威力|h')
    # print('\n'.join(level_up_text_list))

    with open(f'./pokemon/{data["Japanese_name"]}.txt', 'a') as f:
      f.write('**レベルで覚える技\n')
      f.write('|FILTER:T:100%13%CENTER:|32%CENTER|20%CENTER|CENTER:|CENTER:|c\n')
      f.write('|Lv|技名|タイプ|分類|威力|h\n')
      f.write('\n'.join(level_up_text_list))
      f.write('\n')

    # 訓練場の教え技
    move_shop_text_list = []
    if 'Move_Shop_Attacks' in data.keys():
      for attack in data['Move_Shop_Attacks']:
        if len(df_attackdex[df_attackdex['English_name'].str.lower() == attack['English_name'].lower()]):
          s_attackdex = df_attackdex[df_attackdex['English_name'].str.lower() == attack['English_name'].lower()]
          name = s_attackdex['Japanese_name'].values[0]
          type_val = s_attackdex['Type'].values[0]
          category = s_attackdex['Category'].values[0]
          power = s_attackdex['Power'].values[0]
        else:
          name = attack['English_name']
          type_val = ''
          category = ''
          power = ''
        move_shop_text_list.append(f'|{name}|{type_val}|{category}|{power}|')

      # print('**訓練場の教え技')
      # print('|FILTER:T:100%45%CENTER|20%CENTER|20%CENTER:|15%CENTER:|c')
      # print('|技名|タイプ|分類|威力|h')
      # print('\n'.join(move_shop_text_list))

      with open(f'./pokemon/{data["Japanese_name"]}.txt', 'a') as f:
        f.write('**訓練場の教え技\n')
        f.write('|FILTER:T:100%45%CENTER|20%CENTER|20%CENTER:|15%CENTER:|c\n')
        f.write('|技名|タイプ|分類|威力|h\n')
        f.write('\n'.join(move_shop_text_list))
      